In [ ]:
import dlt
from pyspark.sql functions as F

In [ ]:
schema = spark.read.format("csv").option("header",True).option("inferSchema",True).load('/databricks-datasets/airlines/part-00000').schema
airlines_df = spark.read.format("csv").schema(schema).load('/databricks-datasets/airlines/')
display(airlines_df)
airlines_df.printSchema()

In [ ]:
@dlt.create_table(comment="ingest airlanes raw data")
def airlines_raw():
    return(
        spark.read.format("csv").schema(schema).load('/databricks-datasets/airlines/')
        )

In [ ]:
@dlt.create_table(comment="filter rows")
@dlt.expect_or_drop("Origin in Boston", "(Origin == 'BOS')")
@dlt.expect_or_drop("Destination in San Francisco", "(Dest == 'SFO')")
@dlt.expect_or_drop("Only flight in XXI century", "(Year > 2000)")
def bos_sfo_flights():
    return dlt.readStream(airlines_raw)

In [ ]:
@dlt.create_table(comment="filter rows")
@dlt.expect_or_drop("Origin in San Francisco", "(Origin == 'SFO')")
@dlt.expect_or_drop("Destination in Boston", "(Dest == 'BOS')")
@dlt.expect_or_drop("Only flight in XXI century", "(Year > 2000)")
def sfo_bos_flights():
    return dlt.readStream(airlines_raw)

In [ ]:
@dlt.create_table()
def bos_sfo_flights():
    return(
        dlt.read("bos_sfo_flights")
        .union(
            dlt.read("sfo_bos_flights")
        )
    )